## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: string (nullable = true)
-- slots: integer (nullable = true)

Facilities Schema
root
-- facid: integer (nullable = true)
-- name: string (nullable = true)
-- membercost: double (nullable = true)
-- guestcost: double (nullable = true)
-- initialoutlay: integer (nullable = true)
-- monthlymaintenance: integer (nullable = true)

Members Schema
root
-- memid: integer (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: integer (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: integer (nullable = true)
-- joindate: string (nullable = true)

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
SELECT name
FROM facilities
WHERE membercost > 0;

name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
SELECT COUNT(*)
FROM facilities
WHERE membercost = 0;

count(1)
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql
SELECT *
FROM facilities
LIMIT 3;

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
2,Badminton Court,0.0,15.5,4000,50


In [0]:
%sql
SELECT facid, name, membercost, monthlymaintenance
FROM facilities
WHERE membercost > 0
AND membercost < (monthlymaintenance * .20);

facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
SELECT *
FROM facilities
WHERE facid IN (1,5);

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
SELECT name, monthlymaintenance,
CASE 
  WHEN monthlymaintenance <= 100 THEN 'cheap'
  ELSE 'expensive'
END
FROM facilities;

name,monthlymaintenance,CASE WHEN (monthlymaintenance <= 100) THEN cheap ELSE expensive END
Tennis Court 1,200,expensive
Tennis Court 2,200,expensive
Badminton Court,50,cheap
Table Tennis,10,cheap
Massage Room 1,3000,expensive
Massage Room 2,3000,expensive
Squash Court,80,cheap
Snooker Table,15,cheap
Pool Table,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
SELECT *
FROM members
LIMIT 3;

memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,GUEST,GUEST,GUEST,0,(000) 000-0000,null,2012-07-01 00:00:00
1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,null,2012-07-02 12:02:05
2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,null,2012-07-02 12:08:23


In [0]:
%sql
SELECT firstname, surname 
FROM members
WHERE joindate =(
SELECT MAX(joindate) FROM members);

firstname,surname
Darren,Smith


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
SELECT *
FROM bookings
LIMIT 3;

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


In [0]:
%sql
SELECT DISTINCT members.firstname, members.surname, facilities.name,
  CONCAT(firstname, ' ', surname) AS fullname
FROM ((bookings
FULL OUTER JOIN members ON bookings.memid=members.memid)
FULL OUTER JOIN facilities ON bookings.facid=facilities.facid)
WHERE facilities.name LIKE 'Tennis%'
ORDER BY fullname;

firstname,surname,name,fullname
Anne,Baker,Tennis Court 2,Anne Baker
Anne,Baker,Tennis Court 1,Anne Baker
Burton,Tracy,Tennis Court 1,Burton Tracy
Burton,Tracy,Tennis Court 2,Burton Tracy
Charles,Owen,Tennis Court 1,Charles Owen
Charles,Owen,Tennis Court 2,Charles Owen
Darren,Smith,Tennis Court 2,Darren Smith
David,Farrell,Tennis Court 2,David Farrell
David,Farrell,Tennis Court 1,David Farrell
David,Jones,Tennis Court 1,David Jones


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql
SELECT facilities.name, members.firstname, members.surname, facilities.membercost,
  CONCAT(firstname, ' ', surname) AS fullname
FROM ((bookings
FULL OUTER JOIN members ON bookings.memid=members.memid)
FULL OUTER JOIN facilities ON bookings.facid=facilities.facid)
WHERE membercost > 30
UNION
SELECT facilities.name, members.firstname, members.surname, facilities.guestcost,
  CONCAT(firstname, ' ', surname) AS fullname
FROM ((bookings
FULL OUTER JOIN members ON bookings.memid=members.memid)
FULL OUTER JOIN facilities ON bookings.facid=facilities.facid)
WHERE guestcost > 30
AND members.memid = 0
AND starttime BETWEEN '2012-09-14 00:00:00' AND '2012-09-15 00:00:00';

name,firstname,surname,membercost,fullname
Massage Room 1,GUEST,GUEST,80.0,GUEST GUEST
Massage Room 2,GUEST,GUEST,80.0,GUEST GUEST


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql
SELECT facilities.name, members.firstname, members.surname,
  CONCAT(firstname, ' ', surname) AS fullname
FROM ((bookings
FULL OUTER JOIN members ON bookings.memid=members.memid)
FULL OUTER JOIN facilities ON bookings.facid=facilities.facid)
WHERE members.memid IN (SELECT memid FROM bookings
  WHERE memid =0
  AND facilities.guestcost > 30
  AND starttime BETWEEN '2012-09-14 00:00:00' AND '2012-09-15 00:00:00')
OR members.memid IN(SELECT memid FROM bookings
  WHERE memid <> 0
  AND facilities.membercost > 30
  AND starttime BETWEEN '2012-09-14 00:00:00' AND '2012-09-15 00:00:00');

name,firstname,surname,fullname
Massage Room 1,GUEST,GUEST,GUEST GUEST
Massage Room 1,GUEST,GUEST,GUEST GUEST
Massage Room 1,GUEST,GUEST,GUEST GUEST
Massage Room 1,GUEST,GUEST,GUEST GUEST
Massage Room 1,GUEST,GUEST,GUEST GUEST
Massage Room 1,GUEST,GUEST,GUEST GUEST
Massage Room 1,GUEST,GUEST,GUEST GUEST
Massage Room 1,GUEST,GUEST,GUEST GUEST
Massage Room 1,GUEST,GUEST,GUEST GUEST
Massage Room 1,GUEST,GUEST,GUEST GUEST


In [0]:
%sql


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
df_2 = spark.sql('SELECT facilities.name, facilities.monthlymaintenance, facilities.membercost, facilities.guestcost, bookings.facid, bookings.starttime, bookings.memid \
                 FROM bookings FULL OUTER JOIN facilities ON bookings.facid=facilities.facid')
df_2.show(10)



+--------------+------------------+----------+---------+-----+-------------------+-----+
 name|monthlymaintenance|membercost|guestcost|facid| starttime|memid|
+--------------+------------------+----------+---------+-----+-------------------+-----+
Tennis Court 1| 200| 5.0| 25.0| 0|2012-07-04 09:00:00| 2|
Tennis Court 1| 200| 5.0| 25.0| 0|2012-07-04 15:00:00| 2|
Tennis Court 1| 200| 5.0| 25.0| 0|2012-07-06 08:00:00| 0|
Tennis Court 1| 200| 5.0| 25.0| 0|2012-07-06 14:00:00| 0|
Tennis Court 1| 200| 5.0| 25.0| 0|2012-07-06 15:30:00| 2|
Tennis Court 1| 200| 5.0| 25.0| 0|2012-07-07 08:30:00| 2|
Tennis Court 1| 200| 5.0| 25.0| 0|2012-07-07 12:30:00| 0|
Tennis Court 1| 200| 5.0| 25.0| 0|2012-07-07 14:30:00| 2|
Tennis Court 1| 200| 5.0| 25.0| 0|2012-07-08 13:00:00| 3|
Tennis Court 1| 200| 5.0| 25.0| 0|2012-07-08 17:30:00| 2|
+--------------+------------------+----------+---------+-----+-------------------+-----+
only showing top 10 rows

In [0]:
%sql
SELECT facilities.name, SUM(membercost) AS mem_money
FROM bookings FULL OUTER JOIN facilities ON bookings.facid=facilities.facid
WHERE memid > 0
GROUP BY name
UNION
SELECT COUNT(memid) AS num_guest
FROM bookings FULL OUTER JOIN facilities ON bookings.facid=facilities.facid
WHERE memid = 0
GROUP BY name

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-4417278828582813> in <module> 
 ----> 1 df_2 . starttime ( max ) 

 TypeError : 'Column' object is not callable

In [0]:
%sql
SELECT facilities.name, (SUM(membercost) + sub) as revenue
FROM( SELECT SUM(facilities.guestcost)
  FROM bookings FULL OUTER JOIN facilities ON bookings.facid=facilities.facid
  WHERE memid = 0
  GROUP BY facilities.name
  ) sub
WHERE memid > 0
GROUP BY facilities.name
HAVING revenue < 1000;


com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: cannot resolve '`memid`' given input columns: [sub.sum(guestcost)]; line 7 pos 6;
'UnresolvedHaving ('revenue < 1000)
+- 'Aggregate ['facilities.name], ['facilities.name, ('SUM('membercost) + 'sub) AS revenue#1889]
 +- 'Filter ('memid > 0)
 +- SubqueryAlias sub
 +- Aggregate [name#355], [sum(guestcost#357) AS sum(guestcost)#1891]
 +- Filter (memid#333 = 0)
 +- Join FullOuter, (facid#332 = facid#354)
 :- SubqueryAlias spark_catalog.country_club.bookings
 : +- Relation[bookid#331,facid#332,memid#333,starttime#334,slots#335] parquet
 +- SubqueryAlias spark_catalog.country_club.facilities
 +- Relation[facid#354,name#355,membercost#356,guestcost#357,initialoutlay#358,monthlymaintenance#359] parquet

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$$nestedInanonfun$checkAnalysis$2$2.applyOrElse(CheckAnalysis.scala:175)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$$nestedInanonfun$checkAnalysis$2$2.applyOrElse(CheckAnalysis.scala:166)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformUpWithPruning$2(TreeNode.scala:538)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:86)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUpWithPruning(TreeNode.scala:538)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformUpWithPruning$1(TreeNode.scala:535)
	at org.apache.spark.sql.catalyst.expressions.BinaryLikeExpression.mapChildren(Expression.scala:635)
	at org.apache.spark.sql.catalyst.expressions.BinaryLikeExpression.mapChildren$(Expression.scala:634)
	at org.apache.spark.sql.catalyst.expressions.BinaryExpression.mapChildren(Expression.scala:667)
	at org.apache.spark.sql.catalyst.expressions.BinaryExpression.mapChildren(Expression.scala:667)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUpWithPruning(TreeNode.scala:535)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$transformExpressionsUpWithPruning$1(QueryPlan.scala:192)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$mapExpressions$1(QueryPlan.scala:204)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:86)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:204)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.recursiveTransform$1(QueryPlan.scala:215)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$mapExpressions$4(QueryPlan.scala:225)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:322)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:225)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpressionsUpWithPruning(QueryPlan.scala:192)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpressionsUp(QueryPlan.scala:172)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2(CheckAnalysis.scala:166)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2$adapted(CheckAnalysis.scala:99)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:262)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:261)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:261)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:261)
	at org.apache.spark.sql.catalys

In [0]:
%sql
SELECT facilities.name, SUM(facilities.guestcost) as guest_rev
  FROM bookings FULL OUTER JOIN facilities ON bookings.facid=facilities.facid
  WHERE memid = 0
  GROUP BY facilities.name
UNION ALL
SELECT facilities.name, SUM(facilities.membercost) as mem_rev
  FROM bookings FULL OUTER JOIN facilities ON bookings.facid=facilities.facid
  WHERE memid > 0
  GROUP BY facilities.name;

name,guest_rev
Pool Table,265.0
Tennis Court 2,2825.0
Squash Court,4287.5
Tennis Court 1,2500.0
Badminton Court,604.5
Massage Room 1,16640.0
Massage Room 2,6720.0
Table Tennis,90.0
Snooker Table,115.0
Pool Table,0.0


In [0]:
%sql
/*
Attempted multiple times. I understand what I need to do (add the 2 things I am doing a Union on) but cannot figure out the correct way to do it.
*/